In [2]:
# Custom Modules import
from util.data_loader import load_competitions, load_matches
from util.league_table import generate_league_table
from util.elo_calculator import initialize_team_stats, calculate_elo_ratings, loss_from_comparing_tables, elo_grid_search

from statsbombpy import sb
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

%load_ext autoreload 
%autoreload 2

# Nuanced Team Strength Model

### Used Features to quantify Team Stregnth:
- Possession
- Shots on Goal/on Target
- Passes 
- Duels
- 


In [3]:
events = sb.events(match_id=3890259)
events.shape

/Users/mpr/anaconda3/envs/BDEProject/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


(3691, 91)

In [4]:
distinct_types = events['type'].unique()
print(distinct_types)

['Starting XI' 'Half Start' 'Pass' 'Pressure' 'Ball Receipt*' 'Carry'
 'Duel' 'Clearance' 'Ball Recovery' 'Interception' 'Dispossessed'
 'Miscontrol' 'Block' 'Foul Committed' 'Foul Won' 'Shot' 'Goal Keeper'
 'Shield' '50/50' 'Dribble' 'Dribbled Past' 'Injury Stoppage' 'Player Off'
 'Player On' 'Half End' 'Error' 'Substitution' 'Tactical Shift']


In [5]:
# pass_outcome,
result = events.groupby(['team', 'type']).size().reset_index(name='count').sort_values(by='count', ascending=False)
result[result['type']== 'Pass']

,team,type,count
18,Bayern Munich,Pass,797
43,Hamburger SV,Pass,240


In [6]:
print(events['pass_outcome'].unique())

[nan 'Incomplete' 'Out' 'Unknown' 'Pass Offside']


In [8]:
# Calculate completed passes (successful passes with no outcome)
completed_passes = (events[(events['type'] == 'Pass') & 
                         (events['pass_outcome'].isnull())]
                  .groupby('team')
                  .size()
                  .reset_index(name='completed_passes'))

# Calculate total passes
total_passes = (events[events['type'] == 'Pass']
               .groupby('team')
               .size()
               .reset_index(name='total_passes'))

# Merge the two DataFrames
pass_stats = pd.merge(completed_passes, total_passes, on='team')

# Calculate completion percentage
pass_stats['completion_percentage'] = (pass_stats['completed_passes'] / 
                                     pass_stats['total_passes']) * 100

# Sort by completion percentage
pass_stats = pass_stats.sort_values('completion_percentage', ascending=False)

pass_stats.head()

,team,completed_passes,total_passes,completion_percentage
0,Bayern Munich,711,797,89.209536
1,Hamburger SV,150,240,62.500000
